In [ ]:
import utils
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.expand_frame_repr', False)

%matplotlib inline

In [ ]:
path = "results/reward_vector/"
for file in os.listdir(path):
    if os.path.isfile(path + file):
        shape = np.load(path + file, allow_pickle=True).shape
        print(f"{file:<60}     {shape}")

# Scalar Utilities

In [ ]:
path = "results/utility_results/"

results = utils.load_results(path)
keys = sorted(results.keys(), key=utils.sorting)

In [ ]:
intra_wds_table, inter_wds_table = utils.wds_tables(results)
intra_wds_table.pivot(index="Model", columns="Environment", values="WD").head()

In [ ]:
inter_wds_table.pivot(index="Models", columns="Environment", values="WD").head()

In [ ]:
utilities_table = utils.utilities_table(results)
utilities_table.pivot(index="Model", columns="Environment", values="Utility").head()

In [ ]:
for key in keys:
    result_list = [np.mean(result) for result in results[key]]
    print(f"{key:<40} {result_list}")

# Fixed Environment Evaluation

In [ ]:
path = "results/old_fixed_env_results/"

results = utils.load_results(path)
keys = sorted(results.keys(), key=utils.sorting)

unique_table = utils.fixed_env_table(results)

In [ ]:
unique_table.pivot(index="Model", columns="Environment", values="Uniques").head()

In [ ]:
for key in keys:
    uniques = [np.unique(result, axis=0).shape[0] for result in results[key]]
    mean_uniques = np.round(np.mean(uniques), 1)
    print(f"{key:<60} {str(uniques):<40} {mean_uniques}")

# Stale Analysis

In [ ]:
path = "results/legacy_results/"

results_files = {"david": ["david_200221.npy", "david_210221.npy", "david_210221_2.npy"],
                 "my_env": ["my_210220.npy", "my_210221.npy", "my_210221_2.npy"],
                 "new": ["new_gath_repl_1.npy", "new_gath_repl_2.npy", "new_gath_repl_3.npy"],
                 "tfa": ["gathering_replication_1.npy", "gathering_replication_2.npy", "gathering_replication_3.npy",
                         "gathering_replication_4.npy"]


keys = sorted(results_files.keys())
results = dict()
for key in keys:
    results[key] = [np.load(path + file, allow_pickle=True) for file in results_files[key]]

if True:
    for key in keys:
        results[key] = [result[result != -1] for result in results[key]]


In [ ]:
'''
plt.figure(figsize=(15, 12))
i = 1
for result in results["my_env"] + results["david"] + results["tfa"][:-1]:
    plt.subplot(3, 3, i)
    plt.title(f"Number of sampled episodes: {result.shape[0]}")
    plt.hist(result, bins=200, density=True)
    i += 1

plt.tight_layout()
plt.show()
'''
zz =0